In [1]:
import torch
from rfm.nmfhtr import KernelModel
from rfm.kernels import laplacian

# Thiết lập thiết bị
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Tạo dữ liệu ngẫu nhiên
n_samples = 1000  # Số lượng mẫu
n_features = 20   # Số lượng đặc trưng
n_classes = 3     # Số lượng lớp (đầu ra)

# Dữ liệu đầu vào (ngẫu nhiên)
X_train = torch.randn(n_samples, n_features).to(DEVICE)
X_val = torch.randn(n_samples // 10, n_features).to(DEVICE)

# Nhãn đầu ra (ngẫu nhiên, one-hot encoding)
y_train = torch.randint(0, n_classes, (n_samples,)).to(DEVICE)
y_train = torch.nn.functional.one_hot(y_train, num_classes=n_classes).float()

y_val = torch.randint(0, n_classes, (n_samples // 10,)).to(DEVICE)
y_val = torch.nn.functional.one_hot(y_val, num_classes=n_classes).float()

# Khởi tạo KernelModel với kernel Laplace
bandwidth = 1.0  # Tham số bandwidth của kernel Laplace
kernel_model = KernelModel(
    kernel_fn=lambda x, y: laplacian(x, y, bandwidth=bandwidth),
    centers=X_train,
    y_dim=n_classes,
    device=DEVICE
)

# Huấn luyện mô hình
print("Training KernelModel with Laplace kernel...")
# Huấn luyện mô hình
print("Training KernelModel with Laplace kernel...")
results = kernel_model.fit(
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    epochs=10,          # Số lượng epochs
    mem_gb=4,           # Giới hạn bộ nhớ GPU
    n_subsamples=500,   # Số lượng mẫu con để tính toán
    bs=32,              # Batch size
    eta=0.1,            # Learning rate
    classification=True,
    verbose=True
)

# Dự đoán trên tập validation
print("Predicting on validation data...")
predictions = kernel_model.forward(X_val)

# Đánh giá độ chính xác
accuracy = (predictions.argmax(dim=1) == y_val.argmax(dim=1)).float().mean().item()
print(f"Validation Accuracy: {accuracy:.4f}")

c:\Users\hoang\anaconda3\envs\clean_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Training KernelModel with Laplace kernel...
Training KernelModel with Laplace kernel...
Using NMF-based projection with rank=3, eta=0.0031, bs=32


  0%|          | 0/31 [00:00<?, ?it/s]

Using NMF projection
grad shape: torch.Size([33, 3]), kmat shape: torch.Size([33, 500])
W_nmf shape: torch.Size([3, 3]), H_nmf shape: torch.Size([3, 500]), S_nmf shape: torch.Size([3, 3])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (3x500 and 33x3)

In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from rfm import LaplaceRFM, GeneralizedLaplaceRFM, GaussRFM, NTKModel
import torch.nn.functional as F
import logging
import wandb

# Cấu hình logging: ghi log vào file và in ra console
logger = logging.getLogger()
logger.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(message)s')

# File handler: ghi log vào output.txt
file_handler = logging.FileHandler("output.txt")
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

# Console handler: in log ra console
console_handler = logging.StreamHandler()
console_handler.setFormatter(formatter)
logger.addHandler(console_handler)

# Hàm chuyển đổi batch: chuyển đổi nhãn thành one-hot encoding với 10 lớp
def one_hot_collate(batch):
    images, labels = zip(*batch)
    images = torch.stack(images, 0)
    labels = torch.tensor(labels)
    # Chuyển đổi nhãn sang one-hot (float)
    labels = F.one_hot(labels, num_classes=10).float()
    
    # Đảm bảo rằng cả images và labels đều ở trên cùng một thiết bị
    if torch.cuda.is_available():
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)
    
    return images, labels

# Thiết lập thiết bị và bộ nhớ GPU (nếu có)
if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
    DEV_MEM_GB = torch.cuda.get_device_properties(DEVICE).total_memory // 1024**3 - 1
else:
    DEVICE = torch.device("cpu")
    DEV_MEM_GB = 8

# Định nghĩa transform: chuyển đổi ảnh MNIST thành tensor và làm phẳng thành vector 784 chiều
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.view(-1))
])

# Tải dataset MNIST cho training và testing
full_train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Chia tập train ra thành một phần nhỏ để tránh làm đầy bộ nhớ
subset_size = 10000  # Có thể điều chỉnh theo khả năng của máy
train_subset, _ = random_split(full_train_dataset, [subset_size, len(full_train_dataset) - subset_size])

# Tạo DataLoader cho tập train và test, sử dụng collate_fn để chuyển đổi nhãn
batch_size = 16
train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True, collate_fn=one_hot_collate)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=one_hot_collate)

# Tham số cần điều chỉnh
laplace_model = LaplaceRFM(
    bandwidth=1.,  # Nên tune parameter này (thử giá trị 0.5-5)
    device=DEVICE,
    mem_gb=DEV_MEM_GB,
    diag=False
)

# Phần huấn luyện nên sửa thành

logger.info("Training LaplaceRFM")
laplace_model.fit(
    train_data=train_loader,
    test_data=test_loader,
    iters=3,  # Tham số này có thể conflict với epochs
    classification=True,
    total_points_to_sample=subset_size, # Nên để None để dùng toàn bộ data
    M_batch_size=64,  # Tăng batch size để tận dụng GPU
    method='nmf',
    verbose=True,
    epochs=3,  # Nên tăng số epochs (10-50)
)


c:\Users\hoang\anaconda3\envs\clean_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-25 15:46:54,494 - Training LaplaceRFM


Loaders provided
Time taken to prefit nmf with 70000 points: 0.62255859375 seconds


  0%|          | 0/1 [00:00<?, ?it/s]


TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.